In [1]:
import pandas as pd
import re

In [47]:
# Importing as df
df = pd.read_csv('master.csv')
df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [48]:
# Droping columns with high numbers of missing values
df.drop(columns=['HDI for year'], inplace=True)
df.head(15)

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,"2,156,624,900",796,Boomers
5,Albania,1987,female,75+ years,1,35600,2.81,Albania1987,"2,156,624,900",796,G.I. Generation
6,Albania,1987,female,35-54 years,6,278800,2.15,Albania1987,"2,156,624,900",796,Silent
7,Albania,1987,female,25-34 years,4,257200,1.56,Albania1987,"2,156,624,900",796,Boomers
8,Albania,1987,male,55-74 years,1,137500,0.73,Albania1987,"2,156,624,900",796,G.I. Generation
9,Albania,1987,female,5-14 years,0,311000,0.00,Albania1987,"2,156,624,900",796,Generation X


In [49]:
# Delete comma in numbers
df[' gdp_for_year ($) '] = df[' gdp_for_year ($) '].apply(lambda x: int(re.sub(r'[, ]', '', x)))
df.dtypes

country                object
year                    int64
sex                    object
age                    object
suicides_no             int64
population              int64
suicides/100k pop     float64
country-year           object
 gdp_for_year ($)       int64
gdp_per_capita ($)      int64
generation             object
dtype: object

In [51]:
# Calculating modal values for categorical variables
modalValues = df.groupby(['country', 'year', 'age', 'sex', 'generation'])['population'].sum().reset_index()
modalValues = modalValues.groupby(['country', 'year']).\
    apply(lambda x: x.loc[x['population'].idxmax()]).reset_index(drop=True).\
    drop(columns=['population'])
modalValues.head()

,country,year,age,sex,generation
0,Albania,1987,5-14 years,male,Generation X
1,Albania,1988,5-14 years,male,Generation X
2,Albania,1989,5-14 years,male,Generation X
3,Albania,1992,5-14 years,male,Millenials
4,Albania,1993,5-14 years,male,Millenials


In [53]:
# Totalizing values
totalValues = df.groupby(['country', 'year']).\
    agg({'suicides_no': 'sum',
         'population': 'sum',
         'gdp_per_capita ($)': 'mean',
        ' gdp_for_year ($) ': 'mean'}).reset_index()
totalValues[' gdp_for_year ($) '] = totalValues[' gdp_for_year ($) '].astype(int)
totalValues.head()

,country,year,suicides_no,population,gdp_per_capita ($),gdp_for_year ($)
0,Albania,1987,73,2709600,796.0,2156624900
1,Albania,1988,63,2764300,769.0,2126000000
2,Albania,1989,68,2803100,833.0,2335124988
3,Albania,1992,47,2822500,251.0,709452584
4,Albania,1993,73,2807300,437.0,1228071038


In [58]:
# Merging dataframes
dfPro = totalValues.merge(modalValues, on=['country', 'year'], how='left').\
    sort_values(['country', 'year'])
dfPro.head()

,country,year,suicides_no,population,gdp_per_capita ($),gdp_for_year ($),age,sex,generation
0,Albania,1987,73,2709600,796.0,2156624900,5-14 years,male,Generation X
1,Albania,1988,63,2764300,769.0,2126000000,5-14 years,male,Generation X
2,Albania,1989,68,2803100,833.0,2335124988,5-14 years,male,Generation X
3,Albania,1992,47,2822500,251.0,709452584,5-14 years,male,Millenials
4,Albania,1993,73,2807300,437.0,1228071038,5-14 years,male,Millenials


In [57]:
# Creating response variable
dfPro['suicideRate'] = dfPro['suicides_no']*100000/dfPro['population']
dfPro.head()

,country,year,suicides_no,population,gdp_per_capita ($),gdp_for_year ($),age,sex,generation,suicideRate
0,Albania,1987,73,2709600,796.0,2156624900,5-14 years,male,Generation X,2.694125
1,Albania,1988,63,2764300,769.0,2126000000,5-14 years,male,Generation X,2.279058
2,Albania,1989,68,2803100,833.0,2335124988,5-14 years,male,Generation X,2.425886
3,Albania,1992,47,2822500,251.0,709452584,5-14 years,male,Millenials,1.665190
4,Albania,1993,73,2807300,437.0,1228071038,5-14 years,male,Millenials,2.600363


In [ ]:
# selecionar registros modales completos en las mayores partes de los anos, edades, geners, etc
# Encoder
# Crear variables por ventanas
# Dividir
